In [1]:
#!/usr/bin/env python

"""TensorFlow MNIST AutoEncoder

This is my attempt to write the autoencoder for MNIST by Andrej Karpathy using 
ConvNetJS in TensorFlow. Mostly to get some more experience working in 
Tensorflow.

Sources:
    - http://cs.stanford.edu/people/karpathy/convnetjs/demo/autoencoder.html
    - https://www.tensorflow.org/get_started/mnist/pros

Author: Gertjan van den Burg
Date: Thu Oct 26 16:49:29 CEST 2017

"""

import numpy as np
import tensorflow as tf

# from magenta.models.image_stylization.image_utils import form_image_grid
from tensorflow.examples.tutorials.mnist import input_data

BATCH_SIZE = 128
GRID_ROWS = 5
GRID_COLS = 10
USE_RELU = False


def weight_variable(shape):
    # From the mnist tutorial
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fc_layer(previous, input_size, output_size):
    W = weight_variable([input_size, output_size])
    b = bias_variable([output_size])
    return tf.matmul(previous, W) + b

def get_ll(target, output):
    return tf.reduce_sum(
        target * tf.log(output + 1e-10) + \
        (1 - target) * tf.log(1 - output + 1e-10),
        reduction_indices=[-1]
    )


def autoencoder(x):
    # first fully connected layer with 50 neurons using tanh activation
    l1 = tf.nn.tanh(fc_layer(x, 28*28, 256))
    tf.summary.histogram('outputs_l_1', l1)
    # second fully connected layer with 50 neurons using tanh activation
    l2 = tf.nn.tanh(fc_layer(l1, 256, 128))
    tf.summary.histogram('outputs_l_2', l2)
    # third fully connected layer with 2 neurons
    l3 = fc_layer(l2, 128, 2)
    tf.summary.histogram('outputs_l_3', l3)
    # fourth fully connected layer with 50 neurons and tanh activation
    l4 = tf.nn.tanh(fc_layer(l3, 2, 128))
    tf.summary.histogram('outputs_l_4', l4)
    # fifth fully connected layer with 50 neurons and tanh activation
    l5 = tf.nn.tanh(fc_layer(l4, 128, 256))
    tf.summary.histogram('outputs_l_5', l5)
    # readout layer
    if USE_RELU:
        out = tf.nn.relu(fc_layer(l5, 50, 28*28))
    else:
        out = tf.sigmoid(fc_layer(l5, 256, 28*28))
    tf.summary.histogram('outputs_l_6', out)
    # let's use an l2 loss on the output image
    loss = tf.reduce_mean(get_ll(x, out))
    return loss, out, l3

"""
def layer_grid_summary(name, var, image_dims):
    prod = np.prod(image_dims)
    grid = form_image_grid(tf.reshape(var, [BATCH_SIZE, prod]), [GRID_ROWS, 
        GRID_COLS], image_dims, 1)
    return tf.summary.image(name, grid)
"""

def create_summaries(loss, x, latent, output):
    writer = tf.summary.FileWriter("./logs")
    tf.summary.scalar("Loss", loss)
    #layer_grid_summary("Input", x, [28, 28])
    #layer_grid_summary("Encoder", latent, [2, 1])
    #layer_grid_summary("Output", output, [28, 28])
    return writer, tf.summary.merge_all()

"""
def make_image(name, var, image_dims):
    prod = np.prod(image_dims)
    grid = form_image_grid(tf.reshape(var, [BATCH_SIZE, prod]), [GRID_ROWS, 
        GRID_COLS], image_dims, 1)
    s_grid = tf.squeeze(grid, axis=0)

    # This reproduces the code in: tensorflow/core/kernels/summary_image_op.cc
    im_min = tf.reduce_min(s_grid)
    im_max = tf.reduce_max(s_grid)

    kZeroThreshold = tf.constant(1e-6)
    max_val = tf.maximum(tf.abs(im_min), tf.abs(im_max))

    offset = tf.cond(
            im_min < tf.constant(0.0),
            lambda: tf.constant(128.0),
            lambda: tf.constant(0.0)
            )
    scale = tf.cond(
            im_min < tf.constant(0.0),
            lambda: tf.cond(
                max_val < kZeroThreshold,
                lambda: tf.constant(0.0),
                lambda: tf.div(127.0, max_val)
                ),
            lambda: tf.cond(
                im_max < kZeroThreshold,
                lambda: tf.constant(0.0),
                lambda: tf.div(255.0, im_max)
                )
            )
    s_grid = tf.cast(tf.add(tf.multiply(s_grid, scale), offset), tf.uint8)
    enc = tf.image.encode_jpeg(s_grid)

    fwrite = tf.write_file(name, enc)
    return fwrite
"""

def main():
    # initialize the data
    mnist = input_data.read_data_sets('/tmp/MNIST_data')

    # placeholders for the images
    x = tf.placeholder(tf.float32, shape=[None, 784])

    # build the model
    loss, output, latent = autoencoder(x)

    # and we use the Adam Optimizer for training
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(-loss)

    # We want to use Tensorboard to visualize some stuff
    writer, summary_op = create_summaries(loss, x, latent, output)

    first_batch = mnist.train.next_batch(BATCH_SIZE)

    # Run the training loop
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #sess.run(make_image("images/input.jpg", x, [28, 28]), feed_dict={x : 
            #first_batch[0]})
        for i in range(int(200001)):
            batch = mnist.train.next_batch(BATCH_SIZE)
            feed = {x : batch[0]}
            if i % 500 == 0:
                summary, train_loss = sess.run([summary_op, loss], 
                        feed_dict=feed)
                print("step %d, training loss: %g" % (i, train_loss))

                writer.add_summary(summary, i)
                writer.flush()

            #if i % 1000 == 0:
                #sess.run(make_image("images/output_%06i.jpg" % i, output, [28, 
                #    28]), feed_dict={x : first_batch[0]})

            train_step.run(feed_dict=feed)

        # Save latent space
        #pred = sess.run(latent, feed_dict={x : mnist.test._images})
        #pred = np.asarray(pred)
        #pred = np.reshape(pred, (mnist.test._num_examples, 2))
        #labels = np.reshape(mnist.test._labels, (mnist.test._num_examples, 1))
        #pred = np.hstack((pred, labels))
        if USE_RELU:
            fname = "latent_relu.csv"
        else:
            fname = "latent_default.csv"
        #np.savetxt(fname, pred)


if __name__ == '__main__':
    main()

Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training loss: -577.244
step 500, training loss: -213.546
step 1000, training loss: -197.405
step 1500, training loss: -188.104
step 2000, training loss: -176.089
step 2500, training loss: -173.859
step 3000, training loss: -172.342
step 3500, training loss: -172.523
step 4000, training loss: -165.054
step 4500, training loss: -166.145
step 5000, training loss: -160.461
step 5500, training loss: -159.129
step 6000, training loss: -159.977
step 6500, training loss: -150.28
step 7000, training loss: -155.451
step 7500, training loss: -145.669
step 8000, training loss: -152.001
step 8500, training loss: -148.434
step 9000, training loss: -147.297
step 9500, training loss: -151.386
step 10000, training loss: -148.326
step 10500, training loss: -149.35
step 11000, training

KeyboardInterrupt: 

In [ ]:
step 0, training loss: -577.485
step 500, training loss: -202.786
step 1000, training loss: -207.21
step 1500, training loss: -202.004
step 2000, training loss: -187.898
step 2500, training loss: -186.689
step 3000, training loss: -175.804
step 3500, training loss: -172.61
step 4000, training loss: -161.926
step 4500, training loss: -170.936
step 5000, training loss: -158.872
step 5500, training loss: -164.085
step 6000, training loss: -151.848
step 6500, training loss: -159.407
step 7000, training loss: -159.056
step 7500, training loss: -157.305
step 8000, training loss: -158.951
step 8500, training loss: -161.98
step 9000, training loss: -146.214
step 9500, training loss: -154.979
step 10000, training loss: -145.297
step 10500, training loss: -143.071
step 11000, training loss: -150.968
step 11500, training loss: -157.205
step 12000, training loss: -146.146
step 12500, training loss: -144.243
step 13000, training loss: -144.019
step 13500, training loss: -163.697
step 14000, training loss: -150.674
step 14500, training loss: -152.286
step 15000, training loss: -146.486
step 15500, training loss: -157.113
step 16000, training loss: -140.504
step 16500, training loss: -136.147
step 17000, training loss: -140.873
step 17500, training loss: -144.909
step 18000, training loss: -131.302
step 18500, training loss: -147.198
step 19000, training loss: -147.718
step 19500, training loss: -145.261
step 20000, training loss: -139.972
step 20500, training loss: -140.979
step 21000, training loss: -147.93
step 21500, training loss: -140.522
step 22000, training loss: -155.479
step 22500, training loss: -147.228
step 23000, training loss: -139.919
step 23500, training loss: -132.262
step 24000, training loss: -142.463
step 24500, training loss: -130.251
step 25000, training loss: -148.649
step 25500, training loss: -141.536
step 26000, training loss: -140.558
step 26500, training loss: -137.23
step 27000, training loss: -144.847
step 27500, training loss: -135.864
step 28000, training loss: -137.215
step 28500, training loss: -128.116
step 29000, training loss: -139.246
step 29500, training loss: -131.523
step 30000, training loss: -132.965
step 30500, training loss: -137.978
step 31000, training loss: -141.618
step 31500, training loss: -140.294
step 32000, training loss: -132.861
step 32500, training loss: -130.252
step 33000, training loss: -138.328
step 33500, training loss: -142.932
step 34000, training loss: -137.06
step 34500, training loss: -134.373
step 35000, training loss: -131.89
step 35500, training loss: -132.643
step 36000, training loss: -141.806
step 36500, training loss: -147.092
step 37000, training loss: -129.737
step 37500, training loss: -138.656
step 38000, training loss: -134.118
step 38500, training loss: -139.848
step 39000, training loss: -132.592
step 39500, training loss: -125.842
step 40000, training loss: -138.065
step 40500, training loss: -143.349
step 41000, training loss: -141.07
step 41500, training loss: -137.043
step 42000, training loss: -126.963
step 42500, training loss: -136.094
step 43000, training loss: -141.677
step 43500, training loss: -129.368
step 44000, training loss: -131.486
step 44500, training loss: -134.046
step 45000, training loss: -135.94
step 45500, training loss: -131.42
step 46000, training loss: -139.378
step 46500, training loss: -133.611
step 47000, training loss: -137.304
step 47500, training loss: -142.816
step 48000, training loss: -127.18
step 48500, training loss: -129.672
step 49000, training loss: -146.451
step 49500, training loss: -138.04
step 50000, training loss: -132.081
step 50500, training loss: -130.092
step 51000, training loss: -144.464
step 51500, training loss: -139.677
step 52000, training loss: -134.809
step 52500, training loss: -139.341
step 53000, training loss: -131.66
step 53500, training loss: -136.415
step 54000, training loss: -130.769
step 54500, training loss: -133.07
step 55000, training loss: -125.89
step 55500, training loss: -143.225
step 56000, training loss: -131.217
step 56500, training loss: -143.6
step 57000, training loss: -130.617
step 57500, training loss: -143.195
step 58000, training loss: -135.302
step 58500, training loss: -141.117
step 59000, training loss: -137.994
step 59500, training loss: -123.757
step 60000, training loss: -134.097
step 60500, training loss: -134.512
step 61000, training loss: -137.98
step 61500, training loss: -137.607
step 62000, training loss: -124.163
step 62500, training loss: -136.144
step 63000, training loss: -122.271
step 63500, training loss: -129.956
step 64000, training loss: -122.583
step 64500, training loss: -131.837
step 65000, training loss: -145.533
step 65500, training loss: -134.17
step 66000, training loss: -131.014
step 66500, training loss: -128.104
step 67000, training loss: -141.223
step 67500, training loss: -120.874
step 68000, training loss: -142.412
step 68500, training loss: -128.993
step 69000, training loss: -130.299
step 69500, training loss: -120.395
step 70000, training loss: -130.241
step 70500, training loss: -128.016
step 71000, training loss: -124.969
step 71500, training loss: -129.618
step 72000, training loss: -131.069
step 72500, training loss: -140.832
step 73000, training loss: -128.5
step 73500, training loss: -135.724
step 74000, training loss: -131.912
step 74500, training loss: -137.808
step 75000, training loss: -122.381
step 75500, training loss: -134.815
step 76000, training loss: -129.52
step 76500, training loss: -137.294
step 77000, training loss: -129.343
step 77500, training loss: -137.771
step 78000, training loss: -131.339
step 78500, training loss: -126.507
step 79000, training loss: -129.441
step 79500, training loss: -136.201
step 80000, training loss: -127.229
step 80500, training loss: -129.502
step 81000, training loss: -126.975
step 81500, training loss: -130.243
step 82000, training loss: -124.352
step 82500, training loss: -133.099
step 83000, training loss: -135
step 83500, training loss: -128.062
step 84000, training loss: -134.459
step 84500, training loss: -124.606
step 85000, training loss: -125.123
step 85500, training loss: -123.388
step 86000, training loss: -133.21
step 86500, training loss: -129.994
step 87000, training loss: -126.751
step 87500, training loss: -128.521
step 88000, training loss: -129.421
step 88500, training loss: -134.652
step 89000, training loss: -118.857
step 89500, training loss: -127.451
step 90000, training loss: -135.981
step 90500, training loss: -128.619
step 91000, training loss: -127.653
step 91500, training loss: -131.686
step 92000, training loss: -132.896